# Invoke FM (Amazon Titan Model)  for Generating Text (Email Responses) Using Bedrock API

In this lab, you learn how to use Large Language Model (LLM) to generate an email response to a customer who provided negative feedback on the quality of customer service they received from the support engineer. In this notebook, you generate an email with a thank you note based on the customer's previous email. You use the Amazon Titan model using the **Amazon Bedrock API with Boto3 client**.

The prompt used in this task is called a zero-shot prompt. In a zero-shot prompt, you describe the task or desired output to the language model in plain language. The model then uses its pre-trained knowledge and capabilities to generate a response or complete the task based solely on the provided prompt.

#### Scenario
You are an AI/ML Engineer at K21Technologies, and your company receives a significant amount of customer feedback, including both positive and negative responses regarding the support provided by your customer service team. One of your tasks is to generate personalized and empathetic responses to customers who have expressed dissatisfaction, aiming to address their concerns and improve customer satisfaction. Using a large language model (LLM), you need to automate the process of drafting responses based on their sentiment from previous communications.

## 1. Environment setup

In this task, you set up your environment.

In [13]:
# Importing necessary libraries to work with AWS, files, and environment variables
import json  # For working with JSON data
import os  # For interacting with the operating system (e.g., file paths)
import sys  # For modifying system paths

# Importing Boto3 (AWS SDK for Python) and botocore (handles low-level AWS interactions)
import boto3
import botocore

# Adding the parent directory to the system path so we can import files from there
module_path = ".."  # This points to the parent directory
sys.path.append(os.path.abspath(module_path))  # Adds the parent directory to the search path for modules

# Create a connection to Amazon Bedrock service using the Boto3 client
# 'bedrock-runtime' specifies the service we want to connect to
# The region_name is fetched from the environment variable AWS_DEFAULT_REGION
bedrock_client = boto3.client(
    'bedrock-runtime',  # Service name: Bedrock
    region_name=os.environ.get("AWS_DEFAULT_REGION", None)  # AWS region, if available from environment settings
)


### Explanation of the Code:

#### **Importing Libraries:**

- **json**: This library is used to handle **JSON data**, such as parsing or creating JSON files.
- **os**: This module helps interact with the **operating system**, for tasks like accessing file paths or environment variables.
- **sys**: Used for modifying **system paths**, allowing the script to access modules from different locations.

#### **Boto3 and Botocore:**

- **boto3**: This is the **AWS SDK for Python**, which allows you to interact with **AWS services** like Amazon Bedrock.
- **botocore**: This library handles the **low-level interaction** with AWS services, managing tasks such as making requests and receiving responses.

#### **Modifying System Path:**

- The code adds the **parent directory** (`".."`) to the **system path**, enabling the script to access files or modules from that directory.
- **`sys.path.append()`** ensures that Python can find and import the necessary files from the parent folder.

#### **Connecting to Amazon Bedrock:**

- **`boto3.client()`**: This creates a **connection** to Amazon Bedrock using the service name `'bedrock-runtime'`.
- **`region_name`** is fetched from the environment variable **`AWS_DEFAULT_REGION`**, which defines the region to connect to (e.g., `us-east-1`). If not set, it defaults to `None`.
- This step **establishes communication** with the Amazon Bedrock service, allowing you to interact with its **AI models** for tasks like **text generation** or other language-related tasks.


## 2. Generate text

In this task, you prepare an input for the Amazon Bedrock service to generate an email.

In [14]:
# Define the prompt to be sent to the AI model for generating an email

# This is the instruction that tells the model what to do

prompt_data = """
Command: Write an email from Bob, Customer Service Manager, AnyCompany to the customer "John Doe"
who provided negative feedback on the service provided by our customer support
engineer"""


This code is preparing an instruction (known as a prompt) that will be sent to an AI model to generate an email.

In [15]:
# Prepare the data to send to the AI model in JSON format
body = json.dumps({
    "inputText": prompt_data,  # The prompt we created earlier (email request)
    "textGenerationConfig": {  # Settings for generating the text
        "maxTokenCount": 8192,  # Max length of the response
        "stopSequences": [],  # No specific stop point
        "temperature": 0,  # Predictable, less random response
        "topP": 0.9  # Controls how varied the response can be
    }
})


### Explanation of the Code:

This section of the code prepares the **data** to send to the AI model for generating text. The data is formatted in **JSON** format and contains the prompt and configuration settings.

1. **`inputText`**:
   - This is the **prompt** that we created earlier (the request to generate an email). It contains the task for the AI model to perform — in this case, writing an email in response to customer feedback.

2. **`textGenerationConfig`**:
   - This defines the **settings** for how the AI should generate the text. It includes the following parameters:
   
   - **`maxTokenCount`**: This sets the maximum length of the generated text, in terms of tokens. A **token** can be a word or part of a word, so setting it to **8192** allows the model to generate a reasonably long response.
   
   - **`stopSequences`**: This defines any specific **stop points** where the AI should halt generating text. In this case, it's left as an empty list (`[]`), meaning the AI will not stop until it reaches the **end of the prompt** or maximum token count.
   
   - **`temperature`**: This controls the **randomness** of the generated response. A value of `0` means the response will be **predictable** and **deterministic** (i.e., the model will choose the most likely next token). Higher values like `0.7` would make the response more **random**.
   
   - **`topP`**: This parameter controls how **varied** the model's response can be. A value of `0.9` means the model will consider the top **90% of probable next tokens** for generating the response. This helps the model produce more natural and creative outputs while avoiding overly repetitive or irrelevant text.

### Purpose:
The code creates a structured **request** to send to the AI model. The `inputText` provides the instructions for the task (writing the email), while the `textGenerationConfig` ensures that the model generates the response with the desired length, creativity, and predictability. This setup helps guide the AI model in generating the response according to specific requirements.


## 3. Invoke the Amazon Titan Large language model

In this task, you explore how the model generates an output based on the prompt created earlier.

### Complete Output Generation

This email is generated using the Amazon Titan model by understanding the input request and utilizing its inherent understanding of different modalities. The request to the API is synchronous and waits for the entire output to be generated by the model.

In [21]:
import json
import boto3
import botocore

# Bedrock runtime client (supported region)
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

# Nova Lite model
modelId = "amazon.nova-lite-v1:0"
accept = "application/json"
contentType = "application/json"

# 🔑 Use the prompt defined in the ABOVE cell
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [
                {"text": prompt_data}
            ]
        }
    ]
})

try:
    # Invoke Nova Lite
    response = bedrock_client.invoke_model(
        body=body,
        modelId=modelId,
        accept=accept,
        contentType=contentType
    )

    # Parse response
    response_body = json.loads(response["body"].read())

except botocore.exceptions.ClientError as error:
    if error.response["Error"]["Code"] == "AccessDeniedException":
        print(
            f"{error.response['Error']['Message']}\n"
            "https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\n"
            "https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\n"
        )
    else:
        raise error


### Explanation of the Code:

This code interacts with **Amazon Bedrock** to generate text from an AI model, using the **boto3 client** to send requests and handle responses.

1. **Setting Model ID and Request Headers**:
   - **`modelId`**: Specifies the version of the AI model to use (in this case, `'amazon.nova-lite-v1:0'`).
   - **`accept`**: Defines the format of the expected response, which is **JSON**.
   - **`contentType`**: Specifies that the request body will be in **JSON** format.
   - **`outputText`**: Initializes an empty string that will later store the generated response.

2. **Invoking the AI Model**:
   - **`bedrock_client.invoke_model()`** sends the request to the **Amazon Bedrock service** using the specified model ID and the provided data (`body`).
   - The response is expected in **JSON** format, as specified by the **`accept`** and **`contentType`** headers.

3. **Reading and Parsing the Response**:
   - The response from the AI model is received in **JSON** format and is parsed using **`json.loads()`** to convert the response into a Python dictionary.

4. **Extracting the Generated Text**:
   - The **`outputText`** is extracted from the response, which is stored under the **`results`** key. The generated text is retrieved from the first result (`[0]`), specifically from the **`outputText`** field.

5. **Handling Errors**:
   - If an error occurs (such as access being denied), the **`except`** block catches the error.
   - If the error is an **AccessDeniedException**, troubleshooting resources are printed to the console to help resolve the issue.
   - If it’s another type of error, the error is re-raised to be handled elsewhere or logged.

This code prepares the request, sends it to the model, handles potential errors, and extracts the generated text to be used in the application.


In [22]:
# The relevant part of the response starts after the first newline character '\n'.
# Extract the email text by slicing the response string from after the first newline.

email = response_body["output"]["message"]["content"][0]["text"]  # Find the first newline and get everything after it
print(email)  # Print the generated email



Subject: Addressing Your Recent Feedback

Dear John Doe,

I hope this message finds you well.

I am writing to you personally as the Customer Service Manager at AnyCompany, to address the feedback you recently provided regarding the service you experienced with our customer support engineer. I sincerely apologize for any inconvenience or frustration you may have encountered during your interaction with us.

Your satisfaction is of utmost importance to us, and we are committed to providing you with the highest quality of service. It is with a heavy heart that I read your feedback, and I want to assure you that we are taking your concerns very seriously.

To better understand the issues you faced and to ensure they are resolved promptly, I would appreciate it if you could provide more details about your experience. Specifically, it would be helpful to know:

1. The date and time of your interaction with our customer support team.
2. The name of the support engineer you spoke with, if pos

### Explanation of the Code:

This section of the code extracts the **generated email** from the **AI model's response**.

1. **Extracting the Relevant Text**:
   - The **generated text** (`outputText`) includes the entire response from the model, and the **email content** starts after the first newline character (`\n`).
   - **`outputText.index('\n')`** finds the index of the first **newline character** in the response.

2. **Slicing the String**:
   - **`outputText[outputText.index('\n')+1:]`** slices the `outputText` string, starting **just after** the first newline character, effectively extracting the email content.

3. **Printing the Email**:
   - The **email text** is stored in the `email` variable and printed using `print(email)` so that you can view the generated email content.

### Purpose:
This code isolates the **email content** from the model’s response, removing the initial instructions or other text, and prints the **final email** generated by the AI.


### Streaming Output Generation

Bedrock also supports that the output can be streamed as it is generated by the model in form of chunks. This email is generated by invoking the model with streaming option. `invoke_model_with_response_stream` returns a `ResponseStream` which you can read from.

In [41]:
# Initialize an empty list to store the output chunks
output = []
chunk_texts = []  # Temporary storage for grouping text

try:
    # Invoke the model with a response stream to get results as chunks
    response = bedrock_client.invoke_model_with_response_stream(
        body=body,
        modelId=modelId,
        accept=accept,
        contentType=contentType
    )
    stream = response.get('body')

    chunk_number = 1  # Counter for display
    chunk_size = 50   # Group every 50 text pieces into 1 chunk marker (gives 4-5 total chunks)
    text_count = 0    # Count of text pieces
    
    if stream:
        # Process each chunk of data from the stream
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                # Convert the chunk from bytes to a JSON object
                chunk_obj = json.loads(chunk.get('bytes').decode())
                
                # NOVA LITE: Extract text from contentBlockDelta
                if 'contentBlockDelta' in chunk_obj:
                    delta = chunk_obj['contentBlockDelta'].get('delta', {})
                    if 'text' in delta:
                        text = delta['text']
                        output.append(text)
                        chunk_texts.append(text)
                        text_count += 1
                        
                        # Print a chunk marker after every 50 text pieces
                        if text_count % chunk_size == 0:
                            combined_text = ''.join(chunk_texts)
                            print(f'\t\t\x1b[31m**Chunk {chunk_number}**\x1b[0m\n{combined_text}\n')
                            chunk_texts = []
                            chunk_number += 1
        
        # Print any remaining text that didn't reach chunk_size
        if chunk_texts:
            combined_text = ''.join(chunk_texts)
            print(f'\t\t\x1b[31m**Chunk {chunk_number}**\x1b[0m\n{combined_text}\n')

except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\n"
              "To troubleshoot this issue, please refer to the following resources.\n"
              "https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\n"
              "https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
    else:
        raise error


		**Chunk 1**
Subject: Addressing Your Recent Experience with Our Customer Support

Dear John,

I hope this email finds you well.

I am Bob, the Customer Service Manager at AnyCompany, and I am writing to you in response to the feedback you recently provided regarding the service you received from our customer support engineer. First and foremost, I sincerely apologize for any inconvenience or frustration this may have caused you.

At AnyCompany, we strive to provide the highest level of service and support to all our customers, and it is concerning to hear that your experience did not meet your expectations. Your feedback is incredibly valuable to us, as it helps us identify

		**Chunk 2**
 areas for improvement and take corrective actions.

I would like to take this opportunity to understand more about the issues you encountered in detail. Could you please provide further information or specific instances where our service fell short of your expectations? This will help us address th

The stream with response approach helps to quickly obtain the output of the model and allows the service to complete it as you read. This assists in use cases where you request the model to generate longer pieces of text. You can later combine all the chunks generated to form the complete output and use it for your use case.

### Explanation of the Code:

This code interacts with **Amazon Bedrock** using the **response stream** to process and print the model's output in **chunks**.

1. **Initialize an Empty List**:
   - **`output = []`**: Initializes an empty list to store the chunks of the generated text.

2. **Invoking the Model with Response Stream**:
   - The **`invoke_model_with_response_stream()`** method sends the **request** to the AI model and retrieves the **response stream**.
   - The **`body`**, **`modelId`**, **`accept`**, and **`contentType`** are passed to specify the request details.
   - **`stream = response.get('body')`** gets the body of the response, which is a stream of data chunks.

3. **Processing the Stream**:
   - **`i = 1`**: A counter to keep track of the chunks as they are processed.
   - The `for` loop iterates over the **response stream**, and each **chunk** of data is extracted and processed one by one.
   - **`chunk = event.get('chunk')`** retrieves the chunk from the event in the stream.
   - **`json.loads(chunk.get('bytes').decode())`** decodes the chunk from **bytes** to a JSON object.
   - **`text = chunk_obj['outputText']`** extracts the **generated text** (email) from the JSON object.
   - **`output.append(text)`** stores the generated text in the `output` list.
   - **`print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')`** prints the current chunk, with the chunk number (`i`) highlighted in red.

4. **Error Handling**:
   - The **`try`** block ensures the process runs, and the **`except`** block handles any **AWS access errors**.
   - If an **AccessDeniedException** occurs, an error message with troubleshooting links is displayed.
   - Any other errors are re-raised for further handling.

### Purpose:
This code allows **real-time processing of the AI model’s output** in **chunks**, printing each chunk of the generated text as it is received. It helps handle larger outputs efficiently and provides clear feedback for debugging or error handling.


In [42]:
# Combine all the chunks of text into a single string
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')

# Join all the pieces of text from the output list into one complete string
complete_output = ''.join(output)

# Print the complete generated email
print(complete_output)


		**COMPLETE OUTPUT**

Subject: Addressing Your Recent Experience with Our Customer Support

Dear John,

I hope this email finds you well.

I am Bob, the Customer Service Manager at AnyCompany, and I am writing to you in response to the feedback you recently provided regarding the service you received from our customer support engineer. First and foremost, I sincerely apologize for any inconvenience or frustration this may have caused you.

At AnyCompany, we strive to provide the highest level of service and support to all our customers, and it is concerning to hear that your experience did not meet your expectations. Your feedback is incredibly valuable to us, as it helps us identify areas for improvement and take corrective actions.

I would like to take this opportunity to understand more about the issues you encountered in detail. Could you please provide further information or specific instances where our service fell short of your expectations? This will help us address the probl

### Explanation of the Code:

This section of the code combines all the chunks of text that were received in the previous step and prints the **final complete output**.

1. **Printing the Header**:
   - **`print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')`**: This prints a header indicating the start of the final output.
   - The `\x1b[31m` makes the text **red** (for emphasis) and `\x1b[0m` resets the color after the header.

2. **Combining All Chunks into One String**:
   - **`complete_output = ''.join(output)`**: This line combines all the **chunks** of text stored in the `output` list into one **single string**. The `join()` function concatenates each chunk in the list without any spaces in between.
   - **`output`** is a list that contains the individual chunks of text generated by the AI, and **`complete_output`** will hold the final full response.

3. **Printing the Final Output**:
   - **`print(complete_output)`**: This prints the **final generated email**, which is now a single string combining all the chunks.

### Purpose:
This part of the code combines all the text chunks received from the model into a complete, continuous string and prints the **full generated email**. It allows you to see the final output after processing the individual chunks received from the **streaming API**.



You have now experimented with using the boto3 SDK, which provides basic exposure to the Amazon Bedrock API. Using this API, you have seen the use case of generating an email to respond to a customer's negative feedback.

